In [8]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import Calendar
from sqlalchemy import create_engine, text
import pandas as pd

# Configuração de conexão com o banco de dados
DB_USER = 'postgres'
DB_PASSWORD = '1010'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'DB_Final'

class RelatorioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Gerar Relatório por Período")

        # Conectar ao banco de dados
        self.conn = self.connect_db()

        # Criar interface
        self.criar_interface()

    def connect_db(self):
        try:
            conexao_bd = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
            engine = create_engine(
                conexao_bd,
                connect_args={'options': '-csearch_path=sistema,public'}
            )
            conn = engine.connect()
            return conn
        except Exception as e:
            print(f"Erro ao conectar ao banco de dados: {e}")
            return None

    def criar_interface(self):
        # Calendário de início
        tk.Label(self.root, text="Data Início").grid(row=0, column=0)
        self.cal_inicio = Calendar(self.root, selectmode="day", date_pattern="yyyy-mm-dd")
        self.cal_inicio.grid(row=0, column=1)

        # Calendário de fim
        tk.Label(self.root, text="Data Fim").grid(row=1, column=0)
        self.cal_fim = Calendar(self.root, selectmode="day", date_pattern="yyyy-mm-dd")
        self.cal_fim.grid(row=1, column=1)

        # Botão para gerar relatório
        btn_gerar = tk.Button(self.root, text="Gerar Relatório", command=self.gerar_relatorio)
        btn_gerar.grid(row=2, column=0, columnspan=2)

    def gerar_relatorio(self):
        data_inicio = self.cal_inicio.get_date()
        data_fim = self.cal_fim.get_date()

        try:
            # Validar as datas
            if data_inicio > data_fim:
                raise ValueError("A data de início não pode ser maior que a data de fim.")

            # Consultar agendamentos no período
            relatorio_query = text("""
                SELECT A.Data, A.Hora, P_Cliente.Nome as Cliente, P_Medico.Nome as Medico, A.Valor, A.Status
                FROM Agendamento A
                JOIN Cliente C ON A.ID_Cliente = C.ID_Cliente
                JOIN Pessoa P_Cliente ON C.ID_Pessoa = P_Cliente.ID_Pessoa
                JOIN Medico M ON A.ID_Medico = M.ID_Medico
                JOIN Pessoa P_Medico ON M.ID_Pessoa = P_Medico.ID_Pessoa
                WHERE A.Data BETWEEN :data_inicio AND :data_fim
                ORDER BY A.Data, A.Hora
            """)
            resultado = self.conn.execute(relatorio_query, {
                "data_inicio": data_inicio,
                "data_fim": data_fim
            })

            agendamentos = resultado.fetchall()

            # Verificar se há resultados
            if not agendamentos:
                messagebox.showinfo("Relatório", "Nenhum agendamento encontrado no período selecionado.")
                return

            # Exibir resultados em uma tabela (usando pandas para manipular e exibir)
            df = pd.DataFrame(agendamentos, columns=["Data", "Hora", "Cliente", "Médico", "Valor", "Status"])

            # Exibir relatório em uma nova janela
            self.exibir_relatorio(df)

            # Salvar o relatório em um arquivo CSV (opcional)
            # df.to_csv("relatorio_agendamentos.csv", index=False)

        except ValueError as ve:
            messagebox.showerror("Erro", str(ve))
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao gerar o relatório: {e}")

    def exibir_relatorio(self, df):
        # Criar nova janela para exibir o relatório
        relatorio_window = tk.Toplevel(self.root)
        relatorio_window.title("Relatório de Agendamentos")

        # Criar uma Treeview para exibir os dados
        tree = ttk.Treeview(relatorio_window, columns=list(df.columns), show="headings")
        tree.pack(fill=tk.BOTH, expand=True)

        # Definir cabeçalhos
        for col in df.columns:
            tree.heading(col, text=col)
            tree.column(col, width=100)

        # Inserir dados na tabela
        for index, row in df.iterrows():
            tree.insert("", tk.END, values=list(row))

        # Scrollbar para a Treeview
        scrollbar = ttk.Scrollbar(relatorio_window, orient=tk.VERTICAL, command=tree.yview)
        tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Inicialização da interface gráfica
if __name__ == "__main__":
    root = tk.Tk()
    app = RelatorioApp(root)
    root.mainloop()
